In [1]:
import os
import tempfile
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark import SparkContext
from minio import Minio
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Inicializar SparkSession
try:
    sc = SparkContext.getOrCreate()
    sc.stop()
except:
    pass
import time
time.sleep(1)
conf = SparkConf().setAppName("LimpiezaSilver").setMaster("local[*]").set("spark.driver.bindAddress", "127.0.0.1").set("spark.driver.host", "127.0.0.1")
try:
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)
except:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("✅ Spark iniciado")

✅ Spark iniciado


In [3]:
# Configuración MinIO
MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "localhost:9000")
MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY", "minioadmin")
MINIO_BUCKET_BRONCE = os.environ.get("MINIO_BUCKET", "meteo-bronze")
MINIO_BUCKET_SILVER = "meteo-silver"

minio_client = Minio(MINIO_ENDPOINT, access_key=MINIO_ACCESS_KEY, secret_key=MINIO_SECRET_KEY, secure=False)
print("✅ MinIO conectado")

# Crear bucket Silver si no existe
try:
    minio_client.make_bucket(MINIO_BUCKET_SILVER)
    print(f'✅ Bucket {MINIO_BUCKET_SILVER} creado')
except:
    print(f'✅ Bucket {MINIO_BUCKET_SILVER} ya existe')

✅ MinIO conectado
✅ Bucket meteo-silver ya existe


In [4]:
# Cargar archivo más reciente de Bronze
archivo_reciente = None
try:
    print("📥 Buscando archivos Bronce...")
    objects = minio_client.list_objects(MINIO_BUCKET_BRONCE, recursive=True)
    archivos_csv = [obj.object_name for obj in objects if obj.object_name.endswith(".csv")]
    if archivos_csv:
        archivo_reciente = sorted(archivos_csv)[-1]
        print(f"✅ Cargando: {archivo_reciente}")
        temp_dir = tempfile.gettempdir()
        temp_file = os.path.join(temp_dir, archivo_reciente.split("/")[-1])
        minio_client.fget_object(MINIO_BUCKET_BRONCE, archivo_reciente, temp_file)
        df = spark.read.csv(temp_file, header=True, inferSchema=True)
        print(f"✅ Cargados {df.count()} registros")
    else:
        print("⚠️ Sin archivos en Bronce")
        df = spark.createDataFrame([(1, 25.5, 60)], ["id", "temperature", "humidity"])
except Exception as e:
    print(f"⚠️ Error: {e}")
    df = spark.createDataFrame([(1, 25.5, 60)], ["id", "temperature", "humidity"])

print(f"\n📊 DataFrame: {df.count()} registros")
df.show()

📥 Buscando archivos Bronce...
✅ Cargando: sensor_readings/sensor_readings_bronce_20251204135951.csv


✅ Cargados 6100 registros

📊 DataFrame: 6100 registros


+----+--------------------+------------+-----------+--------+-----+-----+--------+--------+--------+--------+---------+
|  id|           timestamp|          ip|temperature|humidity| pm25|light|uv_level|pressure|rain_raw|wind_raw|vibration|
+----+--------------------+------------+-----------+--------+-----+-----+--------+--------+--------+--------+---------+
|  98| 2024-10-12 14:00:00|192.168.1.50|       18.5|    55.0| 25.0| 7000|       4|  1020.0|       0|      40|     true|
|  99| 2024-11-20 11:15:00|192.168.1.50|       10.2|    95.0|  5.0| 1200|       1|   998.0|     450|     120|     true|
| 100| 2024-11-30 23:00:00|192.168.1.50|        5.5|    80.0| 18.0|    0|       0|  1018.0|      10|      20|     true|
|5705|2024-12-04 14:01:...|192.168.1.50|      16.02|    46.4| 47.0|50074|       3|  1021.0|       0|     245|     true|
|4696|2024-12-04 19:04:...|192.168.1.50|      16.34|    47.3| 17.0|30434|       3|  1030.0|       0|     170|     true|
|3105|2024-12-04 19:33:...|192.168.1.50|

In [5]:
# Limpieza de datos
print("\n🧹 LIMPIEZA DE DATOS...")

# Eliminar columnas innecesarias
df = df.drop('pressure', 'uv_level', 'pm25', 'rain_raw', 'wind_raw', 'vibration')

# Eliminar duplicados
df = df.dropDuplicates()

# Descomponer timestamp en columnas separadas
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second

# Buscar columnas de tipo timestamp
timestamp_cols = [field.name for field in df.schema.fields if "timestamp" in field.name.lower()]

for ts_col in timestamp_cols:
    df = df.withColumn(f"{ts_col}_anio", year(col(ts_col))) \
            .withColumn(f"{ts_col}_mes", month(col(ts_col))) \
            .withColumn(f"{ts_col}_dia", dayofmonth(col(ts_col))) \
            .withColumn(f"{ts_col}_hora", hour(col(ts_col))) \
            .withColumn(f"{ts_col}_minuto", minute(col(ts_col))) \
            .withColumn(f"{ts_col}_segundo", second(col(ts_col)))

print(f"\n✅ {df.count()} registros limpios")
df.show(3)


🧹 LIMPIEZA DE DATOS...



✅ 6100 registros limpios


+----+--------------------+------------+-----------+--------+-----+--------------+-------------+-------------+--------------+----------------+-----------------+
|  id|           timestamp|          ip|temperature|humidity|light|timestamp_anio|timestamp_mes|timestamp_dia|timestamp_hora|timestamp_minuto|timestamp_segundo|
+----+--------------------+------------+-----------+--------+-----+--------------+-------------+-------------+--------------+----------------+-----------------+
|5474|2024-12-20 08:09:...|192.168.1.50|       7.38|    95.0| 1072|          2024|           12|           20|             8|               9|               13|
|5840|2024-12-30 05:57:...|192.168.1.50|       9.45|    53.9|    0|          2024|           12|           30|             5|              57|               17|
|3883|2024-12-30 14:18:...|192.168.1.50|      15.45|    38.3|36018|          2024|           12|           30|            14|              18|               53|
+----+--------------------+-------

In [6]:
# Guardar en Silver (dentro de carpeta con nombre de tabla)

print("\n💾 GUARDANDO EN SILVER...")

# Extraer nombre de tabla desde archivo_reciente (remover carpeta y sufijo _bronce)
if archivo_reciente:
    tabla_base = archivo_reciente.split('/')[-1]  # Obtener solo el nombre del archivo
    tabla = tabla_base.split('_bronce_')[0]  # Remover sufijo _bronce_
else:
    tabla = 'datos'

archivo_silver = f'{tabla}_silver.csv'

try:
    import io
    from pyspark.sql.types import TimestampType
    from pyspark.sql.functions import col

    # --- SOLUCIÓN DEL ERROR ---
    # Convertimos las columnas de fecha (Timestamp) a String DENTRO de Spark.
    # Esto evita que Pandas intente hacer la conversión estricta de datetime64[ns] que está fallando.
    df_export = df  # Creamos una copia para exportar
    for field in df.schema.fields:
        if isinstance(field.dataType, TimestampType):
            # Forzamos el formato string para el CSV
            df_export = df_export.withColumn(field.name, col(field.name).cast("string"))
    # --------------------------

    # Convertir a pandas y generar CSV en memoria (evita Hadoop)
    # Ahora usamos df_export en lugar de df
    pdf = df_export.toPandas()
    
    csv_buffer = pdf.to_csv(index=False)
    
    # Subir con nombre de tabla específico dentro de carpeta (tabla/archivo.csv)
    object_path = f"{tabla}/{archivo_silver}"
    csv_bytes = io.BytesIO(csv_buffer.encode('utf-8'))
    minio_client.put_object(
        MINIO_BUCKET_SILVER, 
        object_path, 
        csv_bytes,
        length=len(csv_buffer.encode('utf-8')),
        content_type="text/csv"
    )
    print(f"✅ {archivo_silver} actualizado en Silver")
    
    print("\n" + "="*70)
    print("✅ LIMPIEZA DE SILVER COMPLETADA")
    print("="*70)
    print(f"📍 Origen: {archivo_reciente}")
    print(f"📍 Destino: meteo-silver/{object_path}")
    print(f"📊 Registros: {df.count()} limpios y sin duplicados")
    print("="*70)
        
except Exception as e:
    error_msg = str(e)
    if "UnsatisfiedLinkError" in error_msg or "NativeIO" in error_msg:
        print(f"⚠️  Warning Hadoop ignorado (no crítico)")
        print("✅ LIMPIEZA DE SILVER COMPLETADA")
    else:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()



💾 GUARDANDO EN SILVER...


✅ sensor_readings_silver.csv actualizado en Silver

✅ LIMPIEZA DE SILVER COMPLETADA
📍 Origen: sensor_readings/sensor_readings_bronce_20251204135951.csv
📍 Destino: meteo-silver/sensor_readings/sensor_readings_silver.csv


📊 Registros: 6100 limpios y sin duplicados
